# Style transfer

In [ ]:
import os

try:
    from google.colab import drive
    drive.mount('/content/drive')

    # Change the directory
    os.chdir('/content/drive/MyDrive/ColabNotebooks/style-transfer')
    is_colab = True
    print("Running on Colab")

except:
    is_colab = False
    print("Running locally")


#### Setup

In [ ]:
# 35s
if is_colab:
    ! pip install pretty_midi
    ! apt-get install musescore
    ! sudo apt install lilypond
    ! pip install dfply

    ! pip install pyfluidsynth
    ! apt install musescore fluidsynth

    !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

In [ ]:
#3s
import random
random.seed(42)

from utils.files_utils  import save_pickle, load_pickle, datasets_name, data_path
from utils.plots_utils import plot_train, calculate_TSNEs, plot_tsne
from utils.utils import exp_disponibles, filter_column
from display_audio import save_audios, display_audios, display_score
from roll.guoroll import get_scores_from_roll
from model.train import train_model
from preprocessing import preprocess_data

## Parámetros

In [3]:
#@title ¿Entrenamos un nuevo modelo? - Seleccionar directorios de los datasets
if_train = "Si" #@param ["Si", "No"]

dataset_name = "bach_rag_moz_fres" #@param {type:"string"}
model_name = "bach_rag_moz_fres" #@param {type:"string"}

epoca_inicial =  400#@param {type:"integer"}
entrenar_nuevo = epoca_inicial == 0

epochs =  2#@param {type:"integer"}
checkpt =  1#@param {type:"integer"}

dataset1 = "Bach/" #@param {type:"string"}
dataset2 = "ragtime/" #@param {type:"string"}
dataset3 = "Mozart/" #@param {type:"string"}
dataset4 = "Frescobaldi/" #@param {type:"string"}

songs = {
    dataset1[:-1]: [data_path+dataset1+path for path in os.listdir(data_path+dataset1)],
    dataset2[:-1]: [data_path+dataset2+path for path in os.listdir(data_path+dataset2)],
    dataset3[:-1]: [data_path+dataset3+path for path in os.listdir(data_path+dataset3)],
    dataset4[:-1]: [data_path+dataset4+path for path in os.listdir(data_path+dataset4)],
}

In [ ]:
#@title Transformar estilo
ds_original = "Bach" #@param ["Bach", "ragtime", "Mozart", "Frescobaldi"]
ds_objetivo = "Mozart" #@param ["Bach", "ragtime", "Mozart", "Frescobaldi"]

nombre_pickle = ds_original+"2"+ds_objetivo

____
# Preprocessing and Model

Se lo lleva al formato lista de:

- $n$ canciones = lista de:
- 2 voces = lista de:
- $e$ eventos (notas, silencios, etc.)


Armamos el DataFrame con el que manejamos las canciones. Cada fila es una canción y las columnas son:
- `Autor`
- `Título`
- `Id roll`: qué matriz es de la canción
- `Roll` objeto (GuoRoll) con:
    - `matrix`: matriz de $16*n \times 89$ con $n=$ la cantidad de compases
    - `bars`: cantidad de compases por matriz (es el mismo para todo el dataset, con lo cual, podría eliminarse la redundancia en un trabajo futuro)
    - `song`: referencia al objeto `song` al que pertenece si fue obtenido de ahí. Si fue obtenido desde un embedding, es `None` (en un trabajo futuro podría cambiárselo por un singleton).
    - `score`: partitura obtenida a partir de la matriz
    - `midi`: Pretty MIDI obtenido a partir de la matriz

Luego le agregaremos:
- `Embedding`: el vector resultado de encodear el roll

## Preprocessing

In [4]:
try:
    df_preprocessed = load_pickle(name=dataset_name, path=data_path+"preprocessed_data/")
except:
    df_preprocessed = preprocess_data(songs)
    save_pickle(df_preprocessed, name=dataset_name, path=data_path+"preprocessed_data/")

## Model

In [ ]:
model, callbacks = train_model(
    df=df_preprocessed,
    model_name=model_name,
    new_training=entrenar_nuevo,
    final_epoch=epochs,
    ckpt=checkpt
)

In [ ]:
plot_train(callbacks, epochs)

# Working space

In [ ]:
! git status

In [ ]:
! git add style-transfer.ipynb model/train.py

In [ ]:
# !git commit -m "Running on colab without reload model"
!git push

In [ ]:
!git config --global user.email "lsomacal@gmail.com"
!git config --global user.name "LucasSom"